In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-11-14/diwali_sales_data.csv",
                   encoding="8859")

In [3]:
df.head(10)

,User_ID,Cust_name,Product_ID,Gender,Age Group,Age,Marital_Status,State,Zone,Occupation,Product_Category,Orders,Amount
0,1002903,Sanskriti,P00125942,F,26-35,28,0,Maharashtra,Western,Healthcare,Auto,1,23952.00
1,1000732,Kartik,P00110942,F,26-35,35,1,Andhra Pradesh,Southern,Govt,Auto,3,23934.00
2,1001990,Bindu,P00118542,F,26-35,35,1,Uttar Pradesh,Central,Automobile,Auto,3,23924.00
3,1001425,Sudevi,P00237842,M,0-17,16,0,Karnataka,Southern,Construction,Auto,2,23912.00
4,1000588,Joni,P00057942,M,26-35,28,1,Gujarat,Western,Food Processing,Auto,2,23877.00
5,1000588,Joni,P00057942,M,26-35,28,1,Himachal Pradesh,Northern,Food Processing,Auto,1,23877.00
6,1001132,Balk,P00018042,F,18-25,25,1,Uttar Pradesh,Central,Lawyer,Auto,4,23841.00
7,1002092,Shivangi,P00273442,F,55+,61,0,Maharashtra,Western,IT Sector,Auto,1,NaN
8,1003224,Kushal,P00205642,M,26-35,35,0,Uttar Pradesh,Central,Govt,Auto,2,23809.00
9,1003650,Ginny,P00031142,F,26-35,26,1,Andhra Pradesh,Southern,Media,Auto,4,23799.99


In [4]:
df = df.drop_duplicates().reset_index(drop=True)

In [5]:
df.columns

Index(['User_ID', 'Cust_name', 'Product_ID', 'Gender', 'Age Group', 'Age',
       'Marital_Status', 'State', 'Zone', 'Occupation', 'Product_Category',
       'Orders', 'Amount'],
      dtype='object')

In [10]:
customer_dim = df[["User_ID", 
                   "Cust_name", 
                   "Gender", 
                   "Age", 
                   "Marital_Status", 
                   "Occupation"
                   ]].drop_duplicates().reset_index(drop=True)

In [11]:
age_dim = df[["Age", "Age Group"]].drop_duplicates().reset_index(drop=True)

In [12]:
location_dim = df[["State", "Zone"]].drop_duplicates().reset_index(drop=True)

In [13]:
product_dim = df[["Product_ID", "Product_Category"]].drop_duplicates().reset_index(drop=True)

In [14]:
fact_table = df[["User_ID", "Product_ID", "Amount", "Orders"]].drop_duplicates().reset_index(drop=True)

In [16]:
fact_table["Sale_ID"] = fact_table.index

In [18]:
output = {
    "fact_table": fact_table.to_dict(orient="dict"),
    "customer_dim": customer_dim.to_dict(orient="dict"),
    "age_dim": age_dim.to_dict(orient="dict"),
    "location_dim": location_dim.to_dict(orient="dict"),
    "product_dim": product_dim.to_dict(orient="dict"),
}